In [1]:
import numpy as np
import numpy.ma as ma
import scipy.stats as stat
import random

In [5]:
parameters = {}
parameters["channels"] = ["RED"]
parameters["surface functionalities"] = ["PEG"]
parameters["slices"] = ["S1", "S2", "S3", "S4"]
parameters["videos"] = [1, 2, 3, 4, 5]

channels = parameters["channels"]
surface_functionalities = parameters["surface functionalities"]
slices = parameters["slices"]
videos = parameters["videos"]

geoM2xy = {}
gSEM = {}
SM1x = {}
SM1y = {}
SM2xy = {}
npar = {}

DIR = "E:/Tracking_Videos/Gel_Studies/11_15_17_Gel_Study_37C_72pH/3mM/Output/"
cond = '37C_pH72'

In [6]:
for channel in channels:
    for surface_functionality in surface_functionalities:
        slice_counter = 0
        for slic in slices:
            for video in videos:
                sample_name = "{}_{}_{}_{}_{}".format(channel, surface_functionality, cond, slic, video)                
                #SM2xy[sample_name] = np.genfromtxt('SM2xy_{}.csv'.format(sample_name, delimiter=","))
                
                #npar[sample_name] = SM2xy[sample_name].shape
                geoM2xy[sample_name] = np.genfromtxt(DIR+'geoM2xy_{}.csv'.format(sample_name, delimiter=","))
                gSEM[sample_name] = np.genfromtxt(DIR+'gSEM_{}.csv'.format(sample_name, delimiter=","))

In [7]:
geo_slices = {}
gSEM_slices = {}
w_slices = {}
wo_slices = {}

In [8]:
#Calculate the precision weights over videos
#Alternately, can weight by the number of particles in each video
nvids = 5
nslices = 4
frames = 651

for channel in channels:
    for surface_functionality in surface_functionalities:
        slice_counter = 0
        for slic in slices:
            video_counter = 0
            w_holder = np.zeros((nvids, frames))
            sample_name = "{}_{}_{}_{}".format(channel, surface_functionality, cond, slic)
            for key in geoM2xy:
                if sample_name in key:
                    w_holder[video_counter, :] = 1/(gSEM[key]*gSEM[key])
                    video_counter = video_counter + 1
            wo_slices[sample_name] = np.sum(w_holder, axis=0)
            slice_counter = slice_counter + 1

C:\Users\koolk\Miniconda3\lib\site-packages\ipykernel\__main__.py:16: RuntimeWarning: divide by zero encountered in true_divide


In [ ]:
#Calculate the weights SEMs and means over videos
#Remember to use alternate if not wanting to use precision weights at this level.
for channel in channels:
    for surface_functionality in surface_functionalities:
        slice_counter = 0
        for slic in slices:
            geo_holder = np.zeros((nvids, frames))
            gSEM_holder = np.zeros((nvids, frames))
            w_holder = np.zeros((nvids, frames))
            video_counter = 0
            sample_name = "{}_{}_{}_{}".format(channel, surface_functionality, cond, slic)
            for key in geoM2xy:
                if sample_name in key:
                    w_holder[video_counter, :] = (1/(gSEM[key]*gSEM[key]))/wo_slices[sample_name]
                    geo_holder[video_counter, :] = w_holder[video_counter, :] * geoM2xy[key]
                    gSEM_holder[video_counter, :] = (1/(gSEM[key]*gSEM[key]))
                    video_counter = video_counter + 1
            geo_slices[sample_name] = np.sum(geo_holder, axis=0)
            gSEM_slices[sample_name] = np.sqrt((1/np.sum(gSEM_holder, axis=0)))
            slice_counter = slice_counter + 1

In [ ]:
geo = {}
gS = {}
w_slices = {}
wo_slices = {}

In [ ]:
#Calculate the precision weights over slices
for channel in channels:
    counter = 0
    for surface_functionality in surface_functionalities:
        w_holder = np.zeros((nslices, frames))
        slice_counter = 0
        sample_name = "{}_{}_{}".format(channel, surface_functionality, cond)
        for key in geo_slices:
            if sample_name in key:
                w_holder[slice_counter, :] = 1/(gSEM_slices[key]*gSEM_slices[key])
                slice_counter = slice_counter + 1
        wo_slices[sample_name] = np.sum(w_holder, axis=0)
        counter = counter + 1

In [ ]:
#Calculate the weights SEMs and means over slices
for channel in channels:
    counter = 0
    for surface_functionality in surface_functionalities:
        geo_holder = np.zeros((nslices, frames))
        gSEM_holder = np.zeros((nslices, frames))
        w_holder = np.zeros((nslices, frames))
        slice_counter = 0
        sample_name = "{}_{}_{}".format(channel, surface_functionality, cond)
        for key in geo_slices:
            if sample_name in key:
                w_holder[slice_counter, :] = (1/(gSEM_slices[key]*gSEM_slices[key]))/wo_slices[sample_name]
                geo_holder[slice_counter, :] = w_holder[slice_counter, :] * geo_slices[key]
                gSEM_holder[slice_counter, :] = (1/(gSEM_slices[key]*gSEM_slices[key]))
                slice_counter = slice_counter + 1
        geo[sample_name] = np.sum(geo_holder, axis=0)
        gS[sample_name] = np.sqrt((1/np.sum(gSEM_holder, axis=0)))
        counter = counter + 1

In [ ]:
for key in geo:
    np.savetxt(DIR+'geoM2xy_{}.csv'.format(key), geo[key], delimiter=',')
    np.savetxt(DIR+'gSEM_{}.csv'.format(key), gS[key], delimiter=',')

In [ ]:
SM2xy = np.genfromtxt('SM2xy_RED_nPEG_37C_72pH_S1.csv', delimiter=",")


In [ ]:
SM2xy[10, 100:200]

In [ ]:
x10 = ma.log(SM2xy[10, :]) #take the log of the MSDS at a single timpoint

In [ ]:
x= np.mean(x10) #take the mean of the logs (geometric mean)
print(x)

In [ ]:
y= stat.sem(x10) #find the standard error around the mean of the logs (exponentiate to get confidence intervals)

In [ ]:
random.random()

In [ ]:
#an example, pretending to find weighted averages over 11 videos
SEM = np.zeros([1, 10])
theta = np.zeros([1, 10])
w = np.zeros([1, 10])

#create some fake averages and SEMs for demonstration purposes
for i in range(0, 11):
    theta[0, i - 1] = x + random.random()
    SEM[0, i - 1] = y + random.random()

wi = np.sum(1./(SEM*SEM))

for i in range(0, 11):
    w[0, i -1] = (1/(SEM[0, i-1]*SEM[0, i-1]))/wi


In [ ]:
otheta = np.sum(w*theta)

In [ ]:
oSEM = np.sqrt(1/(np.sum(SEM*SEM)))

In [ ]:
oSEM

In [ ]:
#where real calculations begin, not limited to one row.

#performing averages over an entire dataset of MSDs at all timepoints
geoMSD = np.mean(ma.log(SM2xy), axis=1)

In [ ]:
geoMSD

In [ ]:
gSEM = stat.sem(ma.log(SM2xy), axis=1)

In [ ]:
frames = 651

SEM = np.zeros([frames, 10])
theta = np.zeros([frames, 10])
w = np.zeros([frames, 10])

for i in range(0, 11):
    theta[:, i-1] = geoMSD + np.random.rand(frames)*0.01
    SEM[:, i-1] = gSEM + np.random.rand(frames)*0.0001

wi = np.sum((1./(SEM*SEM)), axis=1)

for i in range(0, 11):
    w[:, i-1] = (1/(SEM[:, i-1]*SEM[:, i-1]))/wi

In [ ]:
otheta = np.sum(w*theta, axis=1)
oSEM = np.sqrt(1/(np.sum(1/(SEM*SEM), axis=1)))

In [ ]:
oetheta = np.exp(otheta)

In [ ]:
loetheta = np.exp(otheta - 1.96*oSEM)
hoetheta = np.exp(otheta + 1.96*oSEM)

In [ ]:
loetheta

In [ ]:
geo